# Potholes detection with additional annotations

- Annotate images on Roboflow
- Export to YOLOv8 format
- Upload to Colab to train with Roboflow dataset

In [ ]:
!pip install fiftyone
!pip install ultralytics
!pip install gradio

## Adding new annotations to the dataset

In [ ]:
# download data from Roboflow
# you can get a URL from https://public.roboflow.com/object-detection/pothole
!curl -L "PUT ROBOFLOW URL HERE" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
# upload `annotations.zip` to Colab and unzip the files
!unzip annotations.zip

In [ ]:
# copy unzip images and annotations to train folder
import shutil
import os.path as op
from glob import glob

for p in glob("annotations/images/*"):
  filename = op.basename(p)
  shutil.copy(p, op.join("train/images", filename))

for p in glob("annotations/labels/*"):
  filename = op.basename(p)
  shutil.copy(p, op.join("train/labels", filename))

## Training Detection Model

Yolov8 - utralytics : https://docs.ultralytics.com/tasks/detect/

**Note** You need to edit `data.yaml`` file from ``../train/images` to

```
train: /content/train/images
val: /content/valid/images
test: /content/test/images
```

on Google Colab.

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.yaml').load('yolov8n.pt')  # build from YAML and transfer weights

# Train the YOLO model for 50 epochs
results = model.train(data="./data.yaml", epochs=30, imgsz=640)

## Run Gradio Application

In [ ]:
import gradio as gr
from PIL import Image, ImageDraw, ImageFont

def inference(gr_input):
    """
    Inference function for gradio.
    """
    pred = model(gr_input)
    draw_prediction = ImageDraw.Draw(gr_input)
    boxes_predict = pred[0].boxes
    boxes = boxes_predict.xyxy.tolist()
    scores = boxes_predict.conf.tolist()
    for score, box in zip(scores, boxes):
        x, y, x2, y2 = tuple(box)
        draw_prediction.rectangle((x, y, x2, y2), outline="red", width=2)
    return gr_input

In [ ]:
imagein = gr.inputs.Image(label="Input Image", type="pil")
imageout = gr.outputs.Image(label="Predicted Image", type="pil")

interface = gr.Interface(
    fn=inference,
    inputs=imagein,
    outputs=imageout,
    title="Potholes detection",
    interpretation="default",
).launch(debug="True")

# launch demo
interface.launch()